In [ ]:
pip install pandas scikit-learn onnx onnxruntime skl2onnx joblib numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
# Medical Department Prediction System
# This system predicts medical departments based on patient symptoms

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib
import onnx
import onnxruntime as ort
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import warnings
warnings.filterwarnings('ignore')

# Step 1: Create synthetic dataset for training
def create_medical_dataset():
    """Create a comprehensive medical dataset for training"""

    # Define symptoms for each department
    department_symptoms = {
        'cardiology': [
            'chest pain', 'shortness of breath', 'palpitations', 'dizziness',
            'fatigue', 'swelling in legs', 'irregular heartbeat', 'high blood pressure',
            'chest tightness', 'rapid heartbeat', 'ankle swelling', 'breathlessness'
        ],
        'neurology': [
            'headache', 'seizures', 'memory loss', 'confusion',
            'numbness', 'tingling', 'weakness', 'tremor',
            'speech difficulties', 'vision problems', 'balance issues', 'dizziness'
        ],
        'orthopedics': [
            'joint pain', 'back pain', 'muscle pain', 'stiffness',
            'fracture', 'swelling', 'limited mobility', 'bone pain',
            'knee pain', 'shoulder pain', 'neck pain', 'hip pain'
        ],
        'pediatrics': [
            'fever', 'cough', 'runny nose', 'ear pain',
            'stomach ache', 'vomiting', 'diarrhea', 'rash',
            'sore throat', 'difficulty sleeping', 'irritability', 'loss of appetite'
        ],
        'general': [
            'fever', 'cold', 'cough', 'body pain',
            'headache', 'fatigue', 'nausea', 'stomach pain',
            'sore throat', 'runny nose', 'weakness', 'loss of appetite'
        ],
        'surgery': [
            'severe abdominal pain', 'trauma', 'wound', 'bleeding',
            'appendix pain', 'gallbladder pain', 'hernia', 'tumor',
            'severe injury', 'surgical site pain', 'post-operative pain', 'acute pain'
        ],
        'psychiatry': [
            'depression', 'anxiety', 'panic attacks', 'mood swings',
            'sleep disorders', 'hallucinations', 'delusions', 'stress',
            'behavioral changes', 'social withdrawal', 'concentration problems', 'suicidal thoughts'
        ],
        'dermatology': [
            'rash', 'itching', 'skin lesions', 'acne',
            'eczema', 'psoriasis', 'skin discoloration', 'hair loss',
            'nail problems', 'skin infection', 'moles', 'skin dryness'
        ],
        'ophthalmology': [
            'vision problems', 'eye pain', 'red eyes', 'blurred vision',
            'double vision', 'light sensitivity', 'eye discharge', 'dry eyes',
            'eye strain', 'floaters', 'night blindness', 'eye injury'
        ],
        'radiology': [
            'internal bleeding', 'fracture detection', 'organ imaging', 'tumor detection',
            'chest imaging', 'abdominal imaging', 'brain imaging', 'bone imaging',
            'contrast studies', 'diagnostic imaging', 'medical scanning', 'internal examination'
        ]
    }

    # Create all possible symptoms list
    all_symptoms = []
    for symptoms in department_symptoms.values():
        all_symptoms.extend(symptoms)
    all_symptoms = list(set(all_symptoms))

    # Generate synthetic data
    data = []
    np.random.seed(42)

    for department, symptoms in department_symptoms.items():
        # Generate 500 samples per department
        for _ in range(500):
            # Create a sample with 4 symptoms
            sample_symptoms = np.random.choice(symptoms, size=4, replace=False)

            # Create feature vector (binary encoding for each symptom)
            feature_vector = [1 if symptom in sample_symptoms else 0 for symptom in all_symptoms]
            feature_vector.append(department)

            data.append(feature_vector)

    # Create DataFrame
    columns = all_symptoms + ['department']
    df = pd.DataFrame(data, columns=columns)

    return df, all_symptoms

# Step 2: Train the model
def train_model():
    """Train Random Forest model for department prediction"""

    print("Creating dataset...")
    df, symptoms_list = create_medical_dataset()

    # Prepare features and labels
    X = df.drop('department', axis=1)
    y = df['department']

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Train Random Forest model
    print("Training Random Forest model...")
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
    rf_model.fit(X_train, y_train)

    # Evaluate model
    y_pred = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Model Accuracy: {:.3f}".format(accuracy))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Save the model and symptoms list
    joblib.dump(rf_model, 'medical_department_model.pkl')
    joblib.dump(symptoms_list, 'symptoms_list.pkl')

    return rf_model, symptoms_list, X_train.columns.tolist()

def convert_to_onnx(model, feature_names):
    """Convert sklearn model to ONNX format with IR version 9 compatibility"""

    print("Converting model to ONNX...")

    # Define input type
    initial_type = [('float_input', FloatTensorType([None, len(feature_names)]))]

    # Convert to ONNX (set opset to 12 and disable ZipMap to avoid dictionary output)
    onnx_model = convert_sklearn(
        model,
        initial_types=initial_type,
        target_opset=12,  # Use a safe opset for IRv9
        options={'zipmap': False}  # Output raw probabilities instead of dict
    )

    # Downgrade IR version to 9 manually (skl2onnx doesn’t expose ir_version directly)
    # ONNX Runtime Java 1.16 supports IR version <= 9
    onnx_model.ir_version = 9

    # Save ONNX model
    with open('medical_department_model.onnx', 'wb') as f:
        f.write(onnx_model.SerializeToString())

    print("ONNX model saved successfully with IR version 9!")
    return onnx_model

# Step 4: Symptom Collection Chatbot
class MedicalChatbot:
    def __init__(self, onnx_model_path, symptoms_list):
        self.session = ort.InferenceSession(onnx_model_path)
        self.symptoms_list = symptoms_list
        self.collected_symptoms = []
        self.symptom_questions = [
            "What is your primary concern or main symptom?",
            "Do you have any other symptoms accompanying this?",
            "Are there any additional symptoms you've noticed?",
            "Any other symptoms or discomfort you're experiencing?"
        ]
        self.current_question = 0

    def extract_symptoms(self, user_input):
        """Extract symptoms from user input using keyword matching"""
        user_input = user_input.lower()
        found_symptoms = []

        # Define symptom mappings for better matching
        symptom_mappings = {
            'cold': 'cold',
            'cough': 'cough',
            'fever': 'fever',
            'headache': 'headache',
            'body pain': 'body pain',
            'bodyache': 'body pain',
            'muscle pain': 'muscle pain',
            'joint pain': 'joint pain',
            'chest pain': 'chest pain',
            'stomach pain': 'stomach pain',
            'stomach ache': 'stomach ache',
            'back pain': 'back pain',
            'sore throat': 'sore throat',
            'runny nose': 'runny nose',
            'nausea': 'nausea',
            'vomiting': 'vomiting',
            'diarrhea': 'diarrhea',
            'fatigue': 'fatigue',
            'weakness': 'weakness',
            'dizziness': 'dizziness',
            'shortness of breath': 'shortness of breath',
            'breathing problem': 'shortness of breath',
            'rash': 'rash',
            'itching': 'itching',
            'vision problem': 'vision problems',
            'blurred vision': 'blurred vision',
            'eye pain': 'eye pain',
            'anxiety': 'anxiety',
            'depression': 'depression',
            'sleep disorder': 'sleep disorders',
            'insomnia': 'sleep disorders'
        }

        # Check for symptom keywords
        for keyword, symptom in symptom_mappings.items():
            if keyword in user_input and symptom in self.symptoms_list:
                if symptom not in found_symptoms:
                    found_symptoms.append(symptom)

        return found_symptoms

    def get_response(self, user_input):
        """Process user input and provide appropriate response"""

        if self.current_question >= 4:
            return self.predict_department()

        # Extract symptoms from user input
        new_symptoms = self.extract_symptoms(user_input)

        if new_symptoms:
            for symptom in new_symptoms:
                if symptom not in self.collected_symptoms:
                    self.collected_symptoms.append(symptom)

            response = "I understand you have: {}. ".format(', '.join(new_symptoms))
            self.current_question += 1

            if len(self.collected_symptoms) >= 4:
                return response + self.predict_department()
            elif self.current_question < 4:
                response += "Currently collected symptoms: {}. ".format(', '.join(self.collected_symptoms))
                response += self.symptom_questions[self.current_question]
                return response
            else:
                return response + self.predict_department()
        else:
            return "I couldn't identify specific symptoms in your response. Could you please describe your symptoms more clearly? For example: fever, cough, headache, body pain, etc."

    def create_feature_vector(self):
        """Create feature vector from collected symptoms"""
        feature_vector = []
        for symptom in self.symptoms_list:
            if symptom in self.collected_symptoms:
                feature_vector.append(1.0)
            else:
                feature_vector.append(0.0)
        return np.array(feature_vector, dtype=np.float32).reshape(1, -1)

    def predict_department(self):
        """Predict medical department based on collected symptoms"""
        if len(self.collected_symptoms) == 0:
            return "No symptoms were identified. Please describe your symptoms clearly."

        # Create feature vector
        features = self.create_feature_vector()

        # Make prediction using ONNX model
        input_name = self.session.get_inputs()[0].name
        prediction = self.session.run(None, {input_name: features})
        predicted_department = prediction[0][0]

        # Get prediction probabilities if available
        try:
            probabilities = prediction[1][0] if len(prediction) > 1 else None
            if probabilities is not None:
                confidence = max(probabilities)
                confidence_str = "{:.2f}".format(float(confidence))
            else:
                confidence_str = "N/A"
        except:
            confidence_str = "N/A"

        response = "\n🏥 MEDICAL DEPARTMENT PREDICTION 🏥\n"
        response += "Based on your symptoms: {}\n".format(', '.join(self.collected_symptoms))
        response += "Recommended Department: {}\n".format(str(predicted_department).upper())
        if confidence_str != "N/A":
            response += "Confidence: {}\n".format(confidence_str)
        response += "\nPlease visit the {} department for proper medical consultation.\n".format(str(predicted_department))
        response += "⚠️ This is an AI prediction for guidance only. Always consult with medical professionals for accurate diagnosis."

        return response

# Step 5: Main execution function
def main():
    """Main function to train model, convert to ONNX, and run chatbot"""

    try:
        # Train the model
        model, symptoms_list, feature_names = train_model()

        # Convert to ONNX
        convert_to_onnx(model, feature_names)

        print("\n" + "="*60)
        print("🤖 MEDICAL SYMPTOM CHATBOT STARTED")
        print("="*60)
        print("Please describe your symptoms. I'll ask up to 4 questions to better understand your condition.")
        print("Type 'quit' to exit the chatbot.\n")

        # Initialize chatbot with ONNX model
        chatbot = MedicalChatbot('medical_department_model.onnx', symptoms_list)

        # Start conversation
        print("Bot: " + chatbot.symptom_questions[0])

        while True:
            user_input = input("\nYou: ").strip()

            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("Bot: Thank you for using the Medical Symptom Chatbot. Take care!")
                break

            if user_input:
                response = chatbot.get_response(user_input)
                print("Bot: {}".format(response))

                # Check if prediction is complete
                if "MEDICAL DEPARTMENT PREDICTION" in response:
                    print("\nWould you like to start a new consultation? (yes/no)")
                    continue_chat = input("You: ").strip().lower()
                    if continue_chat in ['yes', 'y']:
                        chatbot = MedicalChatbot('medical_department_model.onnx', symptoms_list)
                        print("\nBot: " + chatbot.symptom_questions[0])
                    else:
                        break
            else:
                print("Bot: Please provide a valid response.")

    except Exception as e:
        print("An error occurred: {}".format(str(e)))
        print("Please make sure all required libraries are installed:")
        print("pip install pandas scikit-learn onnx onnxruntime skl2onnx joblib numpy")

if __name__ == "__main__":
    main()

# Additional utility functions for web integration
def load_onnx_predictor():
    """Load ONNX model for web application use"""
    try:
        session = ort.InferenceSession('medical_department_model.onnx')
        symptoms_list = joblib.load('symptoms_list.pkl')
        return session, symptoms_list
    except Exception as e:
        print("Error loading model: {}".format(e))
        return None, None

def predict_department_from_symptoms(symptoms_text, session, symptoms_list):
    """Standalone function for department prediction"""
    chatbot = MedicalChatbot('medical_department_model.onnx', symptoms_list)

    # Extract symptoms
    extracted_symptoms = chatbot.extract_symptoms(symptoms_text)
    chatbot.collected_symptoms = extracted_symptoms

    if len(extracted_symptoms) > 0:
        return chatbot.predict_department()
    else:
        return "No recognizable symptoms found in the input text."

# Usage example for web integration:
"""
# For Flask/Django web application:
session, symptoms_list = load_onnx_predictor()
user_symptoms = "I have fever, cough, body pain, and headache"
prediction = predict_department_from_symptoms(user_symptoms, session, symptoms_list)
print(prediction)
"""

Creating dataset...
Training Random Forest model...
Model Accuracy: 1.000

Classification Report:
               precision    recall  f1-score   support

   cardiology       1.00      1.00      1.00       100
  dermatology       1.00      1.00      1.00       100
      general       1.00      1.00      1.00       100
    neurology       1.00      1.00      1.00       100
ophthalmology       1.00      1.00      1.00       100
  orthopedics       1.00      1.00      1.00       100
   pediatrics       1.00      1.00      1.00       100
   psychiatry       1.00      1.00      1.00       100
    radiology       1.00      1.00      1.00       100
      surgery       1.00      1.00      1.00       100

     accuracy                           1.00      1000
    macro avg       1.00      1.00      1.00      1000
 weighted avg       1.00      1.00      1.00      1000

Converting model to ONNX...
ONNX model saved successfully with IR version 9!

🤖 MEDICAL SYMPTOM CHATBOT STARTED
Please describe y

'\n# For Flask/Django web application:\nsession, symptoms_list = load_onnx_predictor()\nuser_symptoms = "I have fever, cough, body pain, and headache"\nprediction = predict_department_from_symptoms(user_symptoms, session, symptoms_list)\nprint(prediction)\n'

In [ ]:
session, symptoms_list = load_onnx_predictor()
user_symptoms = "I have fever, cough, body pain, and headache"
prediction = predict_department_from_symptoms(user_symptoms, session, symptoms_list)
print(prediction)


🏥 MEDICAL DEPARTMENT PREDICTION 🏥
Based on your symptoms: cough, fever, headache, body pain
Recommended Department: GENERAL
Confidence: 0.27

Please visit the general department for proper medical consultation.
⚠️ This is an AI prediction for guidance only. Always consult with medical professionals for accurate diagnosis.


In [ ]:
import joblib
import json

symptoms = joblib.load("/content/symptoms_list.pkl")

with open("symptoms_list.json", "w") as f:
    json.dump(symptoms, f)
